# RECAP - Bu not defterini Google Colab ile açın ;)

In [1]:
import os, sys
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

# 1. Wine review Kaggle challenge'dan verileri yükleyin

### CSV dosyasını indirin (örneğin Kaggle API kullanarak)

In [ ]:
!pip install kaggle --quiet

Ardından, Kaggle hesabınıza gidin ve “Yeni API TOKEN” oluşturun. Bilgisayarınıza kaydedebileceğiniz bir dosyanın indirilmesi başlayacaktır. Şimdi, bu dosyayı aşağıdaki komutla yüklemeniz gerekir:

In [ ]:
from google.colab import files
files.upload();

Bu işlem tamamlandıktan sonra, şunu çalıştırın:

In [ ]:
!mkdir -p ~/.kaggle
!cp "kaggle.json" ~/.kaggle/kaggle.json
# Give read & write rights
!chmod 600 ~/.kaggle/kaggle.json

Artık veri setini aşağıdaki bağlantıdan indirebilirsiniz:

In [ ]:
!kaggle datasets download -d zynicide/wine-reviews

Artık dosyaları açabiliriz.

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('wine-reviews.zip', 'r')
zip_ref.extractall('files')
zip_ref.close()

İndirdiğiniz dosyaları kontrol edelim:

In [ ]:
!ls files

### RAM'e yükleme

In [ ]:
import pandas as pd
import os

data_path = 'files'
df = pd.read_csv(os.path.join(data_path, 'winemag-data-130k-v2.csv'), index_col=0)

In [ ]:
print(df.shape)
df.head(3)

Veri kümesinin her satırı bir şarap tanımına karşılık gelir. Verilerde neler olduğunu, özellikle de farklı sütunları inceleyelim:
- şarap şişesinin fiyatı
- tanımı
- sahip olduğu “puan” sayısı (0 ile 100 arasında bir ölçekte)

Buradaki amaç veri mühendisliği yapmak değildir. Öyleyse, ilgili satırları kaldırarak eksik değerleri halledelim.

In [ ]:
df = df.dropna()
df.shape

# 2. Şişe Açıklamasından Şarap Fiyatını Tahmin Etme

### X,y Oluştur

Eğitim setimizi oluşturun

In [ ]:
X_text = df['description'].values
y = df['price'].values

Bunu kelimelere ayıralım

In [ ]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence
X_words = [text_to_word_sequence(sentence) for sentence in X_text]
X_words[0]

Bir sonraki adım, kelimelerinizi tokenlere dönüştürmektir, çünkü bilgisayar bu tokenlerle çalışacaktır.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tk = Tokenizer()
tk.fit_on_texts(X_words)
X_tokens = tk.texts_to_sequences(X_text)

# Vocab size?
vocab_size = len(tk.word_index)
vocab_size

Şimdi, **verileri doldurmamız** gerekiyor. Önce `sentence_length` değerini kontrol edelim.

In [ ]:
import seaborn as sns
sns.histplot([len(x) for x in X_tokens]);

In [ ]:
# max_length_padding seçin
maxlen = 60

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_pad = pad_sequences(X_tokens, dtype=float, padding='post', maxlen=maxlen)
X_pad.shape

### NLP model

In [ ]:
# Her zaman ortalama fiyatı tahmin ederken temel MAE puanı
y_mean = y.mean()
mae_baseline = np.mean(np.abs(y - y_mean))
print("MAE Baseline:", f"{mae_baseline:.2f}$")

Öncelikle bir NLP modeli oluşturalım. Bu sefer işlevsel API'yi kullanacağız. Sadece eğlence için değil. Nedenini daha sonra göreceksiniz.

In [ ]:
# Embedding boyutu?
embedding_size = 40

In [ ]:
from tensorflow.keras import Model, Input, layers
from tensorflow.keras.optimizers import Adam

# Model girdilerini, katmanlarını ve çıktılarını tanımlayın
inputs_nlp = Input(shape=(maxlen,))
x = layers.Embedding(input_dim=vocab_size+1, output_dim=embedding_size, mask_zero=True)(inputs_nlp)
x = layers.Conv1D(10, kernel_size=15, padding='same', activation="relu")(x)
x = layers.Conv1D(10, kernel_size=10, padding='same', activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dense(30, activation='relu')(x)
x = layers.Dropout(0.15)(x)
outputs_nlp = layers.Dense(1, activation='relu')(x)

# Modeli tanımlayın
model_nlp = Model(inputs=inputs_nlp, outputs=outputs_nlp)

model_nlp.summary()

### TensorBoard'a hoş geldiniz 😎!
Bu, Sinir Ağının nasıl eğitildiğini görmek için harika bir araçtır.

In [ ]:
# TensorBoard notebook uzantısını yükleyin
%load_ext tensorboard

# Önceki çalışmalardan kalan tüm logları temizle
!rm -rf ./logs/

# logları bir klasörde saklayacağız (her fit için bir klasör)
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

Şimdi, fit'lerimizin günlükleri, TensorBoard'un ihtiyaç duyduğu bazı bilgileri depolayan özel olarak tasarlanmış bir klasöre verilecektir.

In [ ]:
from tensorflow.keras.callbacks import TensorBoard
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience=2)

model_nlp.compile(loss="mse", optimizer=Adam(learning_rate=1e-4), metrics=['mae'])

model_nlp.fit(X_pad, y,
          validation_split=0.3,
          epochs=50,
          batch_size=32,
          callbacks=[es, tensorboard_callback]
          )

In [ ]:
%tensorboard --logdir logs/fit

# 3. Sayısal özelliklerden fiyatı tahmin etmek?

Örneğin, her şişenin puanları (0 ila 100 arasında bir ölçekte), bir şarabın ne kadar ucuz veya pahalı olabileceğini bize söylemelidir.

In [ ]:
df.plot.scatter('price', 'points')
plt.xlim(0, 200)

In [ ]:
# Başka hangi kategorik değişkenleri One Hot Encoding ile kodlayabiliriz?
df.nunique()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output=False)
X_region_ohe = ohe.fit_transform(df[['region_2']]) # Buraya diğer sütunları ekleyin

In [ ]:
# Price ve OHE sütunlarını birleştirin
X_num = np.hstack([df[['points']].values, X_region_ohe])
pd.DataFrame(X_num)

In [ ]:
# Her şeyi ölçeklendirin
from sklearn.preprocessing import StandardScaler
X_num = StandardScaler().fit_transform(X_num)
print(X_num.shape)
pd.DataFrame(X_num).head()

In [ ]:
# Model girdilerini, katmanlarını ve çıktılarını tanımlayın
inputs_num = Input(shape=(X_num.shape[1],))
x = layers.Dense(64, activation="relu")(inputs_num)
x = layers.Dense(32, activation="relu")(x)
outputs_num = layers.Dense(1, activation="relu")(x)

# Modeli tanımlayın
model_num = Model(inputs=inputs_num, outputs=outputs_num)

model_num.summary()

In [ ]:
es = EarlyStopping(patience=2)

model_num.compile(loss = "mse", optimizer=Adam(learning_rate=5e-4), metrics=['mae'])
model_num.fit(X_num, y,
          validation_split=0.3,
          epochs=50,
          batch_size=32,
          callbacks=[es]
          )

# 4. Birleştirilmiş RNN + Tablo verileri Girişleri !

RNN ve Tabular veri modelini birleştiren bir model oluşturalım.

Bu nedenle daha önce işlevsel API'yi kullandık: daha gelişmiş model yapıları oluşturmamızı sağlıyor. Bu birleştirilmiş model artık Sıralı bir model değil!

İki modeli oluştururken, hem girişleri hem de çıkışları oluşturduk. Artık bunları birleştirebilir ve üstlerine küçük bir Dense network ekleyebiliriz.

In [ ]:
# Girdileri birleştirin
inputs = [inputs_nlp, inputs_num]

# Çıktıları birleştirin
combined = layers.concatenate([outputs_nlp, outputs_num])

# İki Dense katmanı ekle
x = layers.Dense(10, activation="relu")(combined)
outputs = layers.Dense(1, activation="linear")(x)

# Ve modeli oluşturun
model_combined = Model(inputs=inputs, outputs=outputs)

In [ ]:
model_combined.summary() #🤮

😎 Daha iyi özet

In [ ]:
import tensorflow as tf
tf.keras.utils.plot_model(model_combined, "multi_input_model.png", show_shapes=True)

Şimdi, bunları birleştirip uyarlayabiliriz!

`X` verilerine dikkat edin: hem metin hem de sayısal girdileri içermelidir.

In [ ]:
model_combined.compile(loss="mse", optimizer=Adam(learning_rate=1e-4), metrics=['mae'])
es = EarlyStopping(patience=2)

model_combined.fit(x=[X_pad, X_num],
                   y=y,
                   validation_split=0.3,
                   epochs=100,
                   batch_size=32,
                   callbacks=[es])

**Eğitim sürerken neler olduğuna bir bakalım. Eğitimin oldukça iyi bir MAE ile başladığını görüyor musunuz?**

Çıktıları yeniden başlatmadık: öncekileri yeniden kullandık ve bunlar ayrı modellerde zaten eğitilmişti. Yalnızca derleme ve uyumlama yaptığımız için, eğitim iki alt modelden zaten öğrenilmiş olan ağırlıklardan başlıyor.

Her akış/dal, bir veri giriş kaynağı olarak düşünülebilir. Bu tür verilerle karşılaşabileceğiniz birçok kullanım örneği var mı?

# 5. Bu tür verilere sahip olabileceğiniz herhangi bir örnek var mı?

- Tıbbi veriler: EKG, EEG, MRI, PET, bilişsel değerlendirmeler, biyobelirteçler, ...

<img src="https://wagon-public-datasets.s3.amazonaws.com/data-science-images/DL/medical_data.png" width='70%'>

- Nesne algılama, örneğin otonom araçlarda birçok sensöre (birden fazla kamera, radar, hız, harita vb.) dayalı olarak karar verdiğiniz durumlarda.

<img src="https://wagon-public-datasets.s3.amazonaws.com/data-science-images/DL/autonomous_vehicle.png" width='70%'>
